#### Chat Agent executor with force function calling 
In this experiment, we will be working with building a chat agent using langgraph and using force start function call to ensure the agent calls a particular function at the beginning of its execution

At the start the agent will call a specific tool, then afterwards it can do whatever it wants. 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolExecutor, ToolInvocation
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

In [3]:
tools = [TavilySearchResults(max_results=1)]

In [4]:
tool_executor = ToolExecutor(tools=tools)

In [5]:
model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

In [6]:
from langchain.tools.render import format_tool_to_openai_function

In [7]:
functions = [format_tool_to_openai_function(tool) for tool in tools]
model.bind_functions(functions)

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x127795110>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12778b290>, model_name='mistralai/Mixtral-8x7B-Instruct-v0.1', openai_api_key='fe1f4854dd8970c1d52e05e795d053db950947b1cc4fe010db76f3557f93b3bf', openai_api_base='https://api.together.xyz/v1', openai_proxy=''), kwargs={'functions': [{'name': 'tavily_search_results_json', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}}]})

In [9]:
from typing import Sequence
from langchain_core.messages import BaseMessage

In [10]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [14]:
import json
from langchain_core.messages import FunctionMessage, AIMessage

In [26]:
def should_continue(state: AgentState):
    messages = state['messages']
    last_message:BaseMessage = messages[-1]
    # If there is no function call, then we finish
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

def call_function(state: AgentState):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    action = ToolInvocation(
        tool=fn["name"],
        tool_input=json.loads(fn["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}


def first_model(state: AgentState):
    human_input = state["messages"][-1].content
    ai_message:AIMessage = AIMessage(content="", additional_kwargs={
        "tool_calls": [{ 
            "function": {
                    "name": "tavily_search_results_json",
                    "arguments": json.dumps({"query": human_input})
                }
        }]
    })
    return {"messages": [ai_message]}
        

In [27]:
# define the graph


workflow = StateGraph(AgentState)


workflow.add_node("agent", call_model)
workflow.add_node("action", call_function)
workflow.add_node("starting", first_model) 


workflow.set_entry_point("starting") 

workflow.add_edge("starting", "action")
workflow.add_edge("action", "agent")
workflow.add_conditional_edges("agent", should_continue, {
    "continue": "action",
    "end": END
})

app = workflow.compile()


In [28]:
from langchain_core.messages import HumanMessage

In [29]:
for output in app.stream({"messages": [HumanMessage(content="what is the weather in sf")]}):
    for key, value in output.items():
        print("----")
        print(f"excuting node {key}")
        print(value)
    print("----")

----
excuting node starting
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "what is the weather in sf"}'}}]})]}
----
----
excuting node action
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in San Francisco is {\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1708040070, \'localtime\': \'2024-02-15 15:34\'}, \'current\': {\'last_updated_epoch\': 1708039800, \'last_updated\': \'2024-02-15 15:30\', \'temp_c\': 15.6, \'temp_f\': 60.1, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 4.3, \'wind_kph\': 6.8, \'wind_degree\': 330, \'wind_dir\': \'NNW\', \'pressure_mb\': 1022.0, \'p